<a href="https://colab.research.google.com/github/victorpaolo/py_wine_quality/blob/main/py_machine_learning_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD, RMSprop

In [ ]:
winequality = pd.read_csv('winequality.csv', sep=';')
winequality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  color                 6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [ ]:
del winequality['residual sugar']
del winequality['total sulfur dioxide']
del winequality['color']

In [ ]:
winequality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fixed acidity        6497 non-null   float64
 1   volatile acidity     6497 non-null   float64
 2   citric acid          6497 non-null   float64
 3   chlorides            6497 non-null   float64
 4   free sulfur dioxide  6497 non-null   float64
 5   density              6497 non-null   float64
 6   pH                   6497 non-null   float64
 7   sulphates            6497 non-null   float64
 8   alcohol              6497 non-null   float64
 9   quality              6497 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 507.7 KB


In [ ]:
cols = winequality.columns

In [ ]:
seed = 101
np.random.seed(seed)
scale = StandardScaler()
Xtrain, Xtest, y_train, y_test = train_test_split(winequality.loc[:, cols], winequality.quality.values, test_size=0.20, random_state=101)
X_train = scale.fit_transform(Xtrain)
X_test = scale.transform(Xtest)

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
K = 1

for train, test in kfold.split(X_train, y_train):
    model = Sequential()
    model.add(Dense(128, kernel_initializer='normal',input_dim=10, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(256, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(1))
    #model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    #rmsprop = RMSprop(lr=0.0001)
    #model.compile(optimizer=rmsprop, loss='mse', metrics=['mae'])
    sgd = SGD(lr=0.01, momentum=0.7, decay=0.01, nesterov=False)
    model.compile(optimizer=sgd, loss='mse', metrics=['mae'])
    model.fit(X_train[train], y_train[train], batch_size=16, epochs=30, verbose=1)

    mse_value, mae_value = model.evaluate(X_train[test], y_train[test], verbose=0)
    print('Results from Kfold', K)
    print('-'*30)
    print('MSE {:1.4f}'.format(mse_value))
    print('MAE {:1.4f}'.format(mae_value))
    K += 1

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


217/217 [==============================] - 1s 2ms/step - loss: 0.8083 - mae: 0.3926
Epoch 2/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0636 - mae: 0.1796
Epoch 3/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0532 - mae: 0.1635
Epoch 4/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0474 - mae: 0.1566
Epoch 5/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0438 - mae: 0.1511
Epoch 6/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0437 - mae: 0.1513
Epoch 7/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0420 - mae: 0.1479
Epoch 8/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0412 - mae: 0.1478
Epoch 9/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0391 - mae: 0.1427
Epoch 10/30
217/217 [==============================] - 0s 2ms/step - loss: 0.0382 - mae: 0.1414
Epoch 11/30
217/217 [==============================] - 0s 2m

In [ ]:
from sklearn.metrics import r2_score

print('Final Result:')
print('-'*30)
y_pred = model.predict(X_test)
print('r2 Score:',r2_score(y_test, y_pred))

Final Result:
------------------------------
r2 Score: 0.9920929455868398
